In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity



In [24]:
import pathlib
#pathlib.Path(__file__).parent.resolve()
polls = pd.read_csv("../data/polls_synthetic.csv")
#ratings = pd.read_csv("ratings.csv")
polls


,poll_ID,author_ID,title,option,topic
0,0,1,Which programming language is your favorite?,Python|Java|JavaScript|C++|Ruby,Tech
1,1,1,What's your preferred smartphone brand?,Apple|Samsung|Google|OnePlus|Xiaomi,Tech
2,2,4,How often do you upgrade your computer?,Every year|Every 2 years|Every 3 years|Rarely,Tech
3,3,10,Which tech gadget can't you live without?,Smartphone|Laptop|Smartwatch|Tablet,Tech
4,4,8,Are you excited about the upcoming VR technolo...,Yes|No|Maybe,Tech
...,...,...,...,...,...
154,154,2,How do you like to add greenery to your home?,Indoor plants|Succulents|Flowers|Artificial pl...,Home Décor
155,155,9,What's your go-to home fragrance?,Candles|Diffusers|Incense|Potpourri|Essential ...,Home Décor
156,156,10,What's your favorite type of wall decor?,Paintings|Photographs|Mirrors|Wall decals|Tape...,Home Décor
157,157,7,How often do you change the decor in your home?,Seasonally|Yearly|Rarely|Never,Home Décor


In [23]:
polls_list = polls["topic"].str.get_dummies(sep="|")
# Concatenate the binary genre columns back to the 'movies' DataFrame
polls = pd.concat([polls, polls_list], axis=1)
polls

,poll_ID,author_ID,title,option,topic,Activity,Agriculture,Art,Beauty,Economy,...,Music,Pets,Physics,Politics,Science,Space,Sport,Tech,Technology,Travel
0,0,1,Which programming language is your favorite?,Python|Java|JavaScript|C++|Ruby,Tech,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,1,What's your preferred smartphone brand?,Apple|Samsung|Google|OnePlus|Xiaomi,Tech,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,4,How often do you upgrade your computer?,Every year|Every 2 years|Every 3 years|Rarely,Tech,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,10,Which tech gadget can't you live without?,Smartphone|Laptop|Smartwatch|Tablet,Tech,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,8,Are you excited about the upcoming VR technolo...,Yes|No|Maybe,Tech,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,154,2,How do you like to add greenery to your home?,Indoor plants|Succulents|Flowers|Artificial pl...,Home Décor,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,155,9,What's your go-to home fragrance?,Candles|Diffusers|Incense|Potpourri|Essential ...,Home Décor,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
156,156,10,What's your favorite type of wall decor?,Paintings|Photographs|Mirrors|Wall decals|Tape...,Home Décor,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
157,157,7,How often do you change the decor in your home?,Seasonally|Yearly|Rarely|Never,Home Décor,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# processing of overviews

def process_text(text):
    # replace multiple spaces with one
    text = ' '.join(text.split())
    # lowercase
    text = text.lower()
    return text

movies['genres'] = movies.apply(lambda x: process_text(x.genres),axis=1)

In [43]:
tf_idf = TfidfVectorizer(stop_words="english")
tf_idf_matrix = tf_idf.fit_transform(movies['genres'])

In [48]:
cosine_similarity_matrix = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

In [78]:
def idx_from_title(df, title):
    return df[df["title"]==title].index.values[0]

#idx_from_title(movies, "Sabrina (1995)")

def title_from_idx(df, idx):
    return df[df.index==idx]
title_from_idx(movies, 7)

,movieId,title,genres,count,Action,Adventure,Animation,Children,Comedy,Crime,...,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
7,11,"American President, The (1995)",comedy|drama|romance,70,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [80]:
def recommendations( original_title, df,cosine_similarity_matrix,number_of_recommendations):
    index = idx_from_title(df,original_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = [t[0] for t in similarity_scores_sorted[1:(number_of_recommendations+1)]]
    return df['title'].iloc[recommendations_indices]


In [89]:
#recommendations('Toy Story (1995)', movies, cosine_similarity_matrix, 10)
index = idx_from_title(movies, "Toy Story (1995)")


similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
recommendations_indices = [t[0] for t in similarity_scores_sorted[1:(10+1)]]
movies['title'].iloc[recommendations_indices]


595                          Antz (1998)
753                   Toy Story 2 (1999)
877     Emperor's New Groove, The (2000)
941                Monsters, Inc. (2001)
1206              Shrek the Third (2007)
1409                   Inside Out (2015)
909     Atlantis: The Lost Empire (2001)
1378               The Lego Movie (2014)
902                         Shrek (2001)
572             American Tail, An (1986)
Name: title, dtype: object